In [1]:
!pip install pygame
!pip install mss
!pip install gym
!pip install tqdm

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import gym
import collections
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras import optimizers
from tensorflow.keras import layers

# tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")


In [3]:
with tf.device('/device:GPU:0'):

    from minesweeper_pygame import Minesweeper
    env = Minesweeper(display=False, ROWS = 6, COLS = 6, MINES = 4,OUT='CONDENSED', rewards = {"win" : 1, "loss" : -1, "progress" : 0.9, "noprogress" : -0.3, "YOLO" : -0.3})
    min_state = env.stateConverter(env.get_state()).flatten()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
with tf.device('/device:GPU:0'):

    num_states = env.stateConverter(env.get_state()).shape
    print("Size of State Space ->  {}".format(num_states))


    num_actions = 1
    print("Size of Action Space ->  {}".format(num_actions))

Size of State Space ->  (6, 6, 2)
Size of Action Space ->  1


In [5]:
with tf.device('/device:GPU:0'):

    RANDOM_SEED=1
    np.random.seed(RANDOM_SEED)
    tf.random.set_seed(RANDOM_SEED)

In [6]:
# class OUActionNoise:
#     def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
#         self.theta = theta
#         self.mean = mean
#         self.std_dev = std_deviation
#         self.dt = dt
#         self.x_initial = x_initial
#         self.reset()

#     def __call__(self):
#         # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
#         x = (
#             self.x_prev
#             + self.theta * (self.mean - self.x_prev) * self.dt
#             + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
#         )
#         # Store x into x_prev
#         # Makes next noise dependent on current one
#         self.x_prev = x
#         return x

#     def reset(self):
#         if self.x_initial is not None:
#             self.x_prev = self.x_initial
#         else:
#             self.x_prev = np.zeros_like(self.mean)

In [6]:
with tf.device('/device:GPU:0'):

    class Buffer:
        def __init__(self, buffer_capacity=100000, batch_size=64):

        # Number of "experiences" to store at max
            self.buffer_capacity = buffer_capacity
        # Num of tuples to train on.
            self.batch_size = batch_size

        # Its tells us num of times record() was called.
            self.buffer_counter = 0

        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element

            nstates= num_states[0]*num_states[1]*num_states[2]
            self.state_buffer = np.zeros((self.buffer_capacity, nstates))
            self.action_buffer = np.zeros((self.buffer_capacity, num_actions))
            self.reward_buffer = np.zeros((self.buffer_capacity, 1))
            self.next_state_buffer = np.zeros((self.buffer_capacity, nstates))

    # Takes (s,a,r,s') obervation tuple as input
        def record(self, obs_tuple):
        # Set index to zero if buffer_capacity is exceeded,
        # replacing old records
            index = self.buffer_counter % self.buffer_capacity

            self.state_buffer[index] = obs_tuple[0]
            self.action_buffer[index] = obs_tuple[1]
            self.reward_buffer[index] = obs_tuple[2]
            self.next_state_buffer[index] = obs_tuple[3]

            self.buffer_counter += 1

    # We compute the loss and update parameters
        def learn(self):
        # Get sampling range
            record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
            batch_indices = np.random.choice(record_range, self.batch_size)

        # Convert to tensors
            state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
            action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
            reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
            reward_batch = tf.cast(reward_batch, dtype=tf.float32)
            next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])

        

        # Training and updating Actor & Critic networks.
        # See Pseudo Code.
            with tf.GradientTape() as tape:  #GradientTape is used to find gradient easily

                next_state_batch_converted = tf.reshape(next_state_batch, [self.batch_size, num_states[0], num_states[1], num_states[2]])
                state_batch_converted = tf.reshape(state_batch, [self.batch_size, num_states[0], num_states[1], num_states[2]])
            
                target_actions = target_actor(next_state_batch_converted)
                y = reward_batch + gamma * target_critic([next_state_batch_converted, target_actions])
                critic_value = critic_model([state_batch_converted, action_batch])
                critic_loss = tf.math.reduce_mean(tf.math.square(y - critic_value))

            critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables)
            critic_optimizer.apply_gradients(
                zip(critic_grad, critic_model.trainable_variables)
            )

            with tf.GradientTape() as tape:
                actions = actor_model(state_batch_converted)
                critic_value = critic_model([state_batch_converted, actions]) + 1e-5
            # Used `-value` as we want to maximize the value given
            # by the critic for our actions
                actor_loss = -tf.math.reduce_mean(critic_value)

            actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables)
            actor_optimizer.apply_gradients(
                zip(actor_grad, actor_model.trainable_variables)
            )


# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
    def update_target(tau):
        new_weights = []
        target_variables = target_critic.weights
        for i, variable in enumerate(critic_model.weights):
            new_weights.append(variable * tau + target_variables[i] * (1 - tau))

        target_critic.set_weights(new_weights)

        new_weights = []
        target_variables = target_actor.weights
        for i, variable in enumerate(actor_model.weights):
            new_weights.append(variable * tau + target_variables[i] * (1 - tau))

        target_actor.set_weights(new_weights)

In [7]:
with tf.device('/device:GPU:0'):
    def get_actor():
    # Initialize weights between -3e-3 and 3-e3
#         last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)

        bound = env.ROWS*env.COLS

        inputs = layers.Input(shape=num_states)
        out = Conv2D(18, kernel_size=(5, 5), strides=(1, 1),kernel_initializer=glorot_normal(), activation='relu',padding='same')(inputs)
        out = layers.BatchNormalization()(out)
        out = Conv2D(36, kernel_size=(3, 3), strides=(1, 1),kernel_initializer=glorot_normal(), activation='relu',padding='same')(out)
        out = layers.BatchNormalization()(out)
        out = Flatten()(out)
        out = layers.Dense(288, activation="relu")(out)
        out = layers.BatchNormalization()(out)
        out = layers.Dense(220, activation="relu")(out)
        out = layers.BatchNormalization()(out)
        out = layers.Dense(200, activation="relu")(out)
        out = layers.BatchNormalization()(out)

        outputs = layers.Dense(1, activation="sigmoid")(out)

        outputs = tf.math.add(tf.math.multiply(outputs,bound) ,- 0.5)

        model = tf.keras.Model(inputs, outputs)
        
        return model


    def get_critic():
    # State as input
        state_input = layers.Input(shape=(num_states))
        state_out = Conv2D(18, kernel_size=(5, 5), strides=(1, 1),kernel_initializer=glorot_normal(), activation='relu',padding='same')(state_input)
        state_out = layers.BatchNormalization()(state_out)
        state_out = Conv2D(36, kernel_size=(3, 3), strides=(1, 1),kernel_initializer=glorot_normal(), activation='relu',padding='same')(state_out)
        state_out = layers.BatchNormalization()(state_out)
        state_out = Flatten()(state_out)


    # Action as input
        action_input = layers.Input(shape=(num_actions))
        action_out = layers.Dense(32, activation="relu")(action_input)
        action_out = layers.BatchNormalization()(action_out)

    # Both are passed through seperate layer before concatenating
        concat = layers.Concatenate()([state_out, action_out])

        out = layers.Dense(288, activation="relu")(concat)
        out = layers.BatchNormalization()(out)
        out = layers.Dense(220, activation="relu")(out)
        out = layers.BatchNormalization()(out)
        out = layers.Dense(200, activation="relu")(out)
        out = layers.BatchNormalization()(out)

        outputs = layers.Dense(1)(out)

    # Outputs single value for give state-action
        model = tf.keras.Model([state_input, action_input], outputs)

        return model

    def policy(state):
        
        sampled_actions = tf.squeeze(actor_model(state))
        
        
#     noise = noise_object()
    # Adding noise to action
#     sampled_actions = sampled_actions.numpy() + noise
        sampled_actions = sampled_actions.numpy()

    # We make sure action is within bounds
        legal_action = np.clip(sampled_actions, 0, env.ROWS*env.COLS-1)
        return [np.squeeze(legal_action)]


In [8]:
with tf.device('/device:GPU:0'):

    std_dev = 0.2
# ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))

    actor_model = get_actor()
    critic_model = get_critic()

    target_actor = get_actor()
    target_critic = get_critic()

# Making the weights equal initially
    target_actor.set_weights(actor_model.get_weights())
    target_critic.set_weights(critic_model.get_weights())

# Learning rate for actor-critic models
    critic_lr = 0.001
    actor_lr = 0.001

    critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
    actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

    total_episodes = 8000
# Discount factor for future rewards
    gamma = 0.99
# Used to update target networks
    tau = 0.005

    buffer = Buffer(50000, 64)

In [9]:
actor_model.load_weights('ddpg_actor_6*6_4.h5')
critic_model.load_weights('ddpg_critic_6*6_4.h5')
target_actor.load_weights('ddpg_targetactor_6*6_4.h5')
target_critic.load_weights('ddpg_targetcritic_6*6_4.h5')

In [ ]:
with tf.device('/device:GPU:0'):

# To store reward history of each episode
    ep_reward_list = []
# To store average reward history of last few episodes
    avg_reward_list = []

    from tqdm import tqdm

    for ep in tqdm(range(total_episodes)):

        prev_state = env.reset()
        episodic_reward = 0
        t_step=0
        done = False
    
        if (ep+1)%1000:
            
            print("saving models")

            actor_model.save('ddpg_actor_6*6_4.h5')
            critic_model.save('ddpg_critic_6*6_4.h5')
            target_actor.save('ddpg_targetactor_6*6_4.h5')
            target_critic.save('ddpg_targetcritic_6*6_4.h5')



        while not done and t_step<6*6:

            t_step+=1

            tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)
            
            
#         action = policy(tf_prev_state, ou_noise)
            action = policy(tf_prev_state)
        
        # Recieve state and reward from environment.

            action = np.around(action)
                          
            state, reward, done, _ = env.step(action.astype(int)[0])



            buffer.record((prev_state.flatten(), action, reward, state.flatten()))
            episodic_reward += reward

            buffer.learn()
            update_target(tau)

        # End this episode when `done` is True

            prev_state = state
            

        ep_reward_list.append(episodic_reward)

    # Mean of last 1 episode
        avg_reward = np.mean(ep_reward_list[-1:])
        print("Episode * {} * Avg Reward is ==> {}".format(ep+1, episodic_reward))
        avg_reward_list.append(avg_reward)

# Plotting graph
# Episodes versus Avg. Rewards
    plt.plot(avg_reward_list)
    plt.xlabel("Episode")
    plt.ylabel("Avg. Epsiodic Reward")
    plt.show()

  0%|          | 0/8000 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

  0%|          | 1/8000 [00:05<13:09:07,  5.92s/it]

Episode * 1 * Avg Reward is ==> -10.800000000000004


  0%|          | 2/8000 [00:06<9:54:35,  4.46s/it] 

...

 95%|█████████▍| 7570/8000 [6:33:56<29:51,  4.17s/it]

Episode * 7570 * Avg Reward is ==> -10.800000000000004


 95%|█████████▍| 7571/8000 [6:33:57<22:57,  3.21s/it]

Episode * 7571 * Avg Reward is ==> -1.6


 95%|█████████▍| 7572/8000 [6:34:01<25:28,  3.57s/it]

In [12]:
actor_model

In [16]:
def get_winrate(env,actor):
    games = 0
    moves = 0
    stuck = 0
    won_games = 0
    lost_games = 0
    r = 0
    while games < 1000:
        while True:
            state = env.stateConverter(env.get_state())

            tf_prev_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
            
            action = policy(tf_prev_state)
            action = np.around(action)


            next_state, reward, done, _ = env.step(action.astype(int)[0])

            moves += 1

            if reward == 1:
                won_games += 1
            if reward == -1:
                lost_games += 1

            if done:
                games += 1
                env.reset()
                moves = 0
                break
            elif moves >= 30:
                stuck += 1
                games += 1
                
                lost_games +=1
                env.reset()
                moves = 0
                break
    return(won_games/games)


In [13]:
actor_model = tf.keras.models.load_model('ddpg_actor_6*6_4.h5')

In [17]:
get_winrate(env, actor_model)

0.001